# Model Project - OLG

In [361]:
import numpy as np
from scipy import optimize
import sympy as sm

import matplotlib.pyplot as plt
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description



In this project we will first consider a an OLG model with a pay as you go system which we will later compare to an OLG model with a fully funded system.

Individuals live for two periods and we assume constant population:

$L_t=L_{t+1}$

The lifetime utility of a household is defined as:

$U_t=ln(c_{1t})+\frac{1}{(1+\rho)}ln(c_{2t+1}), \ \ \rho>-1$

This is subject to the following constraints:

$c_{1t}=w_t(1-\tau)-s_t$

$c_{2t+1}=s_t(1+r_{t+1})+\tau w_{t+1}$

Thus the model has the following parameters:

$L_t$: The population at time $t$

$U_t$: Lifetime household utility

$\rho$: Consumption discount rate

$c_{1t}$: Consumption of the young

$c_{2t+1}$: Consumption of the old

$w_t$: Wage

$r_t$: Return on capital

$s_t$: Savings

In [362]:
# Define symbols for sympy
K_t = sm.symbols('K_t')
L_t = sm.symbols('L_t')
A = sm.symbols('A')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
rho = sm.symbols('rho')
w_t = sm.symbols('w_t')
r_t = sm.symbols('r_t')
k_t = sm.symbols('k_t')
k_plus = sm.symbols('k_t+1')
Pi = sm.symbols('Pi_t')

s_t = sm.symbols('s_t')
U_t = sm.symbols('U_t')
U = sm.symbols('U_t')
r_plus = sm.symbols('r_t+1')
k = sm.symbols('k^*')

d_t = sm.symbols('d_t')
d_plus = sm.symbols('d_t+1')
tau = sm.symbols('t')
s_payg = sm.symbols('s_payg')
w_plus = sm.symbols('w_t+1')
k_payg = sm.symbols('k_payg')
U_payg = sm.symbols('U_paygt')
kpaygss = sm.symbols('k_payg^*')

In [363]:
#In order to derive the wage (w) and return on capital (r) we need to solve the firms profit maximization problem.

#Define the profit function
profit = A*(K_t**alpha)*(L_t**(1-alpha))-w_t*L_t-(r_t+delta)*K_t

#Setup the FOC's
foc1=sm.diff(profit,K_t)
foc2=sm.diff(profit,L_t)

#Solve FOC's
solve_foc1=sm.solve(foc1,r_t)[0]
solve_foc2=sm.solve(foc2,w_t)[0]

#Print eqations
print('The profit of a given firm is defined as:')
display(sm.Eq(Pi,profit))
print('By taking the first order conditions to this, we get the expressions for w an r:')
display(sm.Eq(r_t,solve_foc1))
display(sm.Eq(w_t,solve_foc2))

The profit of a given firm is defined as:


Eq(Pi_t, A*K_t**alpha*L_t**(1 - alpha) - K_t*(delta + r_t) - L_t*w_t)

By taking the first order conditions to this, we get the expressions for w an r:


Eq(r_t, A*K_t**alpha*L_t*L_t**(-alpha)*alpha/K_t - delta)

Eq(w_t, A*K_t**alpha*L_t**(-alpha)*(1 - alpha))

We now assume full capital depreciation $\delta=1$

In [364]:
#Setup equations
sample_r=alpha*A*k_t**(alpha-1)-1
sample_w=(1-alpha)*A*k_t**alpha

#Display them
print('By using the marginal products of capital and labour we can simplify these expressions to:')
display(sm.Eq(r_t,sample_r))
display(sm.Eq(w_t,sample_w))

By using the marginal products of capital and labour we can simplify these expressions to:


Eq(r_t, A*alpha*k_t**(alpha - 1) - 1)

Eq(w_t, A*k_t**alpha*(1 - alpha))

We now move unto the household optimization problem and determine the savings rate.

In [365]:
#We solve the household optimization problem and determine savings (s)

#Substitute constraints into utility function
U_t = sm.log(w_t*(1-tau)-s_t)+(1/(1+rho))*sm.log(s_t*(1+r_plus)+tau*w_plus)

#Take the FOC in regards to savings
FOC_U = sm.diff(U_t,s_t)

#Isolate savings
isolate_s = sm.solve(FOC_U,s_t)[0]

#Print results
print('Substitute the constraints into the utility function')
display(sm.Eq(U,U_t))
print('Take the FOC in regards to savings:')
display(sm.Eq(0,FOC_U))
print('Isolate savings:')
display(sm.Eq(s_t,isolate_s))

Substitute the constraints into the utility function


Eq(U_t, log(-s_t + w_t*(1 - t)) + log(s_t*(r_t+1 + 1) + t*w_t+1)/(rho + 1))

Take the FOC in regards to savings:


Eq(0, (r_t+1 + 1)/((rho + 1)*(s_t*(r_t+1 + 1) + t*w_t+1)) - 1/(-s_t + w_t*(1 - t)))

Isolate savings:


Eq(s_t, (-r_t+1*t*w_t + r_t+1*w_t - rho*t*w_t+1 - t*w_t - t*w_t+1 + w_t)/(r_t+1*rho + 2*r_t+1 + rho + 2))

# Steady state

We now wish to determine the steady state level of capital $k^*$

In [366]:
# We define capital accumulation using the previuos expressions and solve for the steady state level of capital.

#capital accumulation
display(sm.Eq(k_plus,isolate_s))
ca = sm.Eq(k_plus,isolate_s)

#Steady state capital
ssk = sm.solve(ca,k_t)[0]

#Print equation
print('The steady state level of capital is defined as:')
display(sm.Eq(k,ssk))

Eq(k_t+1, (-r_t+1*t*w_t + r_t+1*w_t - rho*t*w_t+1 - t*w_t - t*w_t+1 + w_t)/(r_t+1*rho + 2*r_t+1 + rho + 2))

IndexError: list index out of range

## Analytical solution

In [360]:
#Create a function to determine capital level in steady state given a set of parameters
ss_func = sm.lambdify((alpha,n,rho,A),ssk)

# Assign arbitrary values to the parameters
value_alpha = 1/4
value_n = 0.025
value_rho = 0.06
value_A = 1

#Run the function with the values
res=ss_func(value_alpha,value_n,value_rho,value_A)

#Print the results
print('Given the parameters we find the steady state level of capital to be:')
display(sm.Eq(k,round(res,ndigits=4)))

SyntaxError: invalid syntax (<lambdifygenerated-41>, line 1)

We could also try running the same model but with no population growth

In [323]:
# Assign values again, but this time n=0
value2_alpha = 1/4
value2_n = 0
value2_rho = 0.06
value2_A = 1

#Run the function with the values
res2=ss_func(value2_alpha,value2_n,value2_rho,value2_A)

#Print the results
print('With no pop growth we find the steady state level of capital to be:')
display(sm.Eq(k,round(res2,ndigits=4)))

With no pop growth we find the steady state level of capital to be:


Eq(k^*, 0.26)

We can use **sympy** to find an analytical expression for the steady state, i.e. solve

$$ \tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] $$

First we define all **symbols**:

In [324]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

Then we define the **steady state equation**

In [325]:
ss = sm.Eq(k,(s*k**alpha+(1-delta)*k)/((1+n)*(1+g)))

and **solve** it

In [326]:
kss = sm.solve(ss,k)[0]
kss

((delta + g*n + g + n)/s)**(1/(alpha - 1))

For later use, we turn the solution into a **Python funciton**

In [327]:
ss_func = sm.lambdify((s,g,n,alpha,delta),kss)

## Numerical solution

We can re-write the equation for the steady state capital per capita as

\\[ 0 = \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] - \tilde{k}^{\ast} \\]

whereby it clearly becomes a **root-finding problem**. Such a problem can be solved by a **bisection method**.

Define the model **parameters**:

In [328]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

**Solve numerically** for the steady state:

In [329]:
solution = modelproject.solve_for_ss(s,g,n,alpha,delta)

print(f'analytical solution is: {ss_func(s,g,n,alpha,delta):.3f}')
print(f' numerical solution is: {solution.root:.3f}')

NameError: name 'modelproject' is not defined

# Further analysis

ADD FURTHER ANALYSIS, VISUALIZATIONS AND EXTENSIONS.

# Conclusion

ADD CONCISE CONCLUSION.